In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import math
import lasio
import os
import pandas as pd

path_to_RIGIS = r'./welldata/RIGIS/'
path_to_GIS = r'./welldata/GIS_md/'
path_to_welltracks = r'./welldata/welltracks.txt'

def getLasDataFrames(path = path_to_RIGIS):
    lasDF = []
    for file in os.listdir(path):
        if file.endswith('.las'):
            lasDF.append(lasio.read(path+file).df())
    return lasDF

lasDataFrame = getLasDataFrames()

In [ ]:
len(lasDataFrame)

In [ ]:
def parse_welltracks():
    welltracks = []
    f = open(path_to_welltracks)
    welldata = f.read().split(';')
    for well in welldata:
        if len(well.split()) > 2:
            name = well.split()[2][:-1]
            data = well.split()[3:]
            x = [float(data[i]) for i in range(len(data)) if i % 4 == 0]
            y = [float(data[i]) for i in range(len(data)) if i % 4 == 1]
            tvdss = [float(data[i]) for i in range(len(data)) if i % 4 == 2]
            md = [float(data[i]) for i in range(len(data)) if i % 4 == 3]

            welltracks.append([name, x, y ,tvdss, md]) # в таком виде хранятся

    return welltracks


In [ ]:
from scipy.interpolate import interp1d
import math
def get_heights_list():
    welltracks = parse_welltracks()
    tvdconv = []
    for tracknum in range(len(welltracks)):
        f = interp1d(welltracks[tracknum][4], welltracks[tracknum][3])
        tvdconv.append(f)

    heights = []
    print(len(lasDataFrame))
    for k  in range(len(lasDataFrame)):
        frame = lasDataFrame[k]
        tmp_sum = 0
        for i in range(len(frame['KG_T']) - 1):
            if (frame['KG_T'].array[i + 1] >= 0):
                tmp_sum += tvdconv[k](frame['KG_T'].index[i + 1]) - tvdconv[k](frame['KG_T'].index[i])
        heights.append(tmp_sum)
    return heights
get_heights_list()

In [28]:
def dist(x1,y1,x2,y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)
def perimeter(boundary):
    (prev_x,prev_y) = boundary[-1]
    perim = 0
    for (x,y) in boundary:
        perim += dist(x,y,prev_x,prev_y)
        prev_x, prev_y = x,y
    return perim
def getCommonLength(boundary1, boundary2):
    len = 0
    print(type(boundary1))
    print((boundary1.size))
    print((boundary2.size))
    for i in range(int(boundary1.size/2)):
        for j in range(int(boundary2.size/2)):
            (x1, y1) = boundary1[i]
            (x2, y2) = boundary2[j]
            if(x1 == x2 and y1== y2):
                print(i, ' and ', j, ' neibors')
    return len
    


In [30]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import contextily as ctx
from shapely import geometry
from shapely.ops import cascaded_union
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords
from main import parse_welltracks

welltracks = parse_welltracks()
boundary = np.load("boundary.npy",'r') 

c = np.array([(track[1][0], track[2][0]) for track in welltracks])
s = geometry.Polygon(boundary)
region_polys, region_pts = voronoi_regions_from_coords(c, s)
perimeters = []
boundaries = []
new_polys = []
for i in range(len(region_polys)):
    #print(type(region_polys[i]))
    try:
        new_polys.append(region_polys[i])
        perimeters.append(perimeter(np.array(region_polys[i].boundary)))
        boundaries.append(region_polys[i].boundary)
    except Exception:
        print()

#print(perimeters)
#fig, ax = subplot_for_map(figsize=(18, 16))
#plot_voronoi_polys_with_points_in_area(ax, s, region_polys, c, region_pts)
adj_mat = []
print(np.array(boundaries).size)
adj_mat.append([0 for i in boundaries]) # i
adj_mat.append([0 for i in boundaries]) # j
for i in range((np.array(boundaries)).size//2):
    for j in range((np.array(boundaries)).size//2):
        if i == j:
            adj_mat[i][j] = 0
            continue
        adj_mat[i][j] = getCommonLength(np.array(boundaries[i]), np.array(boundaries[j]))
print(adj_mat)



#plt.show()


/home/evgeny/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3398: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_8101/3724776862.py:24: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  perimeters.append(perimeter(np.array(region_polys[i].boundary)))
/tmp/ipykernel_8101/3724776862.py:24: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  perimeters.append(perimeter(np.array(region_polys[i].boundary)))
/tmp/ipykernel_8101/3724776862.py:24: FutureWarning: The input object of type 'LineString'


76
<class 'numpy.ndarray'>
926
694
0  and  344  neibors
1  and  343  neibors
462  and  344  neibors
<class 'numpy.ndarray'>
926
14
0  and  3  neibors
461  and  2  neibors
462  and  3  neibors
<class 'numpy.ndarray'>
926
12
<class 'numpy.ndarray'>
926
10
<class 'numpy.ndarray'>
926
338
<class 'numpy.ndarray'>
926
18
<class 'numpy.ndarray'>
926
438
<class 'numpy.ndarray'>
926
12
<class 'numpy.ndarray'>
926
396
<class 'numpy.ndarray'>
926
860
<class 'numpy.ndarray'>
926
12
<class 'numpy.ndarray'>
926
16
<class 'numpy.ndarray'>
926
12
<class 'numpy.ndarray'>
926
560
<class 'numpy.ndarray'>
926
14
<class 'numpy.ndarray'>
926
424
460  and  3  neibors
461  and  2  neibors
<class 'numpy.ndarray'>
926
110
<class 'numpy.ndarray'>
926
12
<class 'numpy.ndarray'>
926
466
<class 'numpy.ndarray'>
926
10
<class 'numpy.ndarray'>
926
10
<class 'numpy.ndarray'>
926
390
<class 'numpy.ndarray'>
926
504
<class 'numpy.ndarray'>
926
14
<class 'numpy.ndarray'>
926
44
<class 'numpy.ndarray'>
926
14
<class 'num

IndexError: list index out of range